<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/evaluation/semantic_similarity_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embedding Similarity Evaluator

This notebook shows the `SemanticSimilarityEvaluator`, which evaluates the quality of a question answering system via semantic similarity.

Concretely, it calculates the similarity score between embeddings of the generated answer and the reference answer.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
# !pip install llama-index

In [2]:
from jet.llm.ollama import initialize_ollama_settings
initialize_ollama_settings()

_Settings(_llm=Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x11483d5e0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x17aa01ee0>, completion_to_prompt=<function default_completion_to_prompt at 0x17ab24a40>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='llama3.1', temperature=0.0, context_window=4096, request_timeout=300.0, prompt_key='prompt', json_mode=False, additional_kwargs={}, is_function_calling_model=True, keep_alive=None), _embed_model=OllamaEmbedding(model_name='nomic-embed-text', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x11483d5e0>, num_workers=None, base_url='http://localhost:11434', ollama_additional_kwargs={}), _callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x11483d5e0>, _tokenizer=None, _node_parser=None, _prompt_he

In [3]:
from llama_index.core.evaluation import SemanticSimilarityEvaluator

evaluator = SemanticSimilarityEvaluator()

In [4]:
# This evaluator only uses `response` and `reference`, passing in query does not influence the evaluation
# query = 'What is the color of the sky'

response = "The sky is typically blue"
reference = """The color of the sky can vary depending on several factors, including time of day, weather conditions, and location.

During the day, when the sun is in the sky, the sky often appears blue. 
This is because of a phenomenon called Rayleigh scattering, where molecules and particles in the Earth's atmosphere scatter sunlight in all directions, and blue light is scattered more than other colors because it travels as shorter, smaller waves. 
This is why we perceive the sky as blue on a clear day.
"""

result = await evaluator.aevaluate(
    response=response,
    reference=reference,
)

In [5]:
print("Score: ", result.score)
print("Passing: ", result.passing)  # default similarity threshold is 0.8

Score:  0.8409488879332032
Passing:  True


In [6]:
response = "Sorry, I do not have sufficient context to answer this question."
reference = """The color of the sky can vary depending on several factors, including time of day, weather conditions, and location.

During the day, when the sun is in the sky, the sky often appears blue. 
This is because of a phenomenon called Rayleigh scattering, where molecules and particles in the Earth's atmosphere scatter sunlight in all directions, and blue light is scattered more than other colors because it travels as shorter, smaller waves. 
This is why we perceive the sky as blue on a clear day.
"""

result = await evaluator.aevaluate(
    response=response,
    reference=reference,
)

In [7]:
print("Score: ", result.score)
print("Passing: ", result.passing)  # default similarity threshold is 0.8

Score:  0.4570148774635365
Passing:  False


### Customization

In [10]:
from llama_index.core.evaluation import SemanticSimilarityEvaluator
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.base.embeddings.base import SimilarityMode

local_embed_model = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = resolve_embed_model(f"local:{local_embed_model}")
evaluator = SemanticSimilarityEvaluator(
    embed_model=embed_model,
    similarity_mode=SimilarityMode.DEFAULT,
    similarity_threshold=0.6,
)

In [11]:
response = "The sky is yellow."
reference = "The sky is blue."

result = await evaluator.aevaluate(
    response=response,
    reference=reference,
)

In [12]:
print("Score: ", result.score)
print("Passing: ", result.passing)

Score:  0.6470747655095268
Passing:  True


We note here that a high score does not imply the answer is always correct.  

Embedding similarity primarily captures the notion of "relevancy". Since both the response and reference discuss "the sky" and colors, they are semantically similar.